In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sqlite3
from sqlalchemy import create_engine

import datetime

from statsmodels.tsa.seasonal import seasonal_decompose, STL

In [30]:
ccy='BTCUSDT'
data_frequency='1d'
start_date='2022-06-01'
end_date='2022-12-31'

In [31]:
def get_and_process_data(ccy, data_frequency, start_date, end_date):
    try:
        
        # Get the data
        db_address = 'sqlite:///../../data/db/crypto.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_all_features" + "_" + ccy + "_" + data_frequency
        
        sql_command = "SELECT * FROM " + input_tbl_name

        if(start_date!="" and end_date!=""):
            
            sd = datetime.datetime.strptime(start_date,'%Y-%m-%d').strftime('%Y-%m-%d')
            ed = datetime.datetime.strptime(end_date,'%Y-%m-%d').strftime('%Y-%m-%d')

            sql_append = " WHERE " + "date(date) >= " + "'" + sd + "'" + " AND date(date) <= " + "'" + ed + "'"
            sql_command = sql_command + sql_append

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        # Converting Date Column to DateTime Type
        # Set the index on the dataframe
        df['date_index'] = pd.to_datetime(df['date'])
        df.set_index('date_index', inplace=True)

        return df

    except Exception as e:
        print(e)

In [32]:
df = get_and_process_data(ccy, data_frequency, start_date, end_date)

In [33]:
df_simple = df[['close']]

In [34]:
df_simple_decomposed = seasonal_decompose(df_simple, model='additive')

In [35]:
fig = go.Figure()

fig.add_trace(go.Scatter(name='trend', x=df.index, y=df_simple_decomposed.trend, mode='lines'))
fig.add_trace(go.Scatter(name='close', x=df.index, y=df.close, mode='lines'))

fig.update_layout(title="", width=1200, height=800)

fig.show()